In [162]:
def flow_minicost(G,K,alpa,bet,gam):
    #Minimizing the cost of fractional multicomodity flow

    from sage.numerical.mip import MixedIntegerLinearProgram, MIPSolverException

    # initialize a minimization program
    p = MixedIntegerLinearProgram(maximization=False)
    Kk = range(len(K))
    sk = np.zeros(10)
    tk = np.zeros(10)
    bk = np.zeros(10)
    for k in range(0,10):
        sk[k], tk[k], bk[k] = K[k]
    print(bk)
    # declaring variables
    x = p.new_variable(real= True, nonnegative = True, name='x')
    z = p.new_variable(binary=True, name='z') #if the arc is used at least one request
    y = p.new_variable(binary=True, name = 'y') #a pair of arcs uv and vu is used
    c = p.new_variable(integer = True, nonnegative = True, name = 'c') #capacity
    # add constraints:
    for k in range(0,10):
        for u, v, label in G.edges():
            p.add_constraint(x[u,v,k] <= z[u,v]*100)
    for u,v, label in G.edges():
        p.add_constraint(0<= z[u,v] + z[v,u] - 2*y[u,v])
        p.add_constraint(z[u,v] + z[v,u] - 2*y[u,v] <=1)
    for k in Kk:
        for u in G.vertices():
            if (u==sk[k]):
                p.add_constraint(sum(x[u,v,k] for v in G.neighbors(u)) - sum(x[v,u,k] for v in G.neighbors(u)) == bk[k])
            elif (u==tk[k]):
                p.add_constraint(sum(x[u,v,k] for v in G.neighbors(u)) - sum(x[v,u,k] for v in G.neighbors(u)) == -bk[k])
            else:
                p.add_constraint(sum(x[u,v,k] for v in G.neighbors(u)) - sum(x[v,u,k] for v in G.neighbors(u)) == 0)
    for u,v, label in G.edges():
        p.add_constraint(sum(x[u,v,k] for k in Kk) <= c[0])

    #add objective to minimize

    #p.set_objective(sum(z[u,v] for u, v, label in G.edges())*alpa + sum(sum(x[u,v,k] for u, v, label in G.edges()) for k in Kk)*bet - sum(y[u,v] for u, v, label in G.edges())*gam*0.5)
    p.set_objective(c[0])
    #try:
    p.solve()
    #except MIPSolverException:
        #print("the problem has no feasible solution")
        #return []

    c_result = p.get_values(c)
    return c_result

In [163]:
import numpy as np

G = graphs.CycleGraph(10)
G.add_edges([(1, 3), (4, 9), (5, 8)])
G = DiGraph(G)
K = [(0, 1, 1), (0, 5, 2), (0, 6, 1), (1, 2, 1), (2, 9, 3), (1, 6, 2), (2, 7, 1), (2, 8, 1), (6, 5, 1), (8, 1, 2)]
alpa = 100
bet = 5
gam = 20
c_result = flow_minicost(G,K,alpa,bet,gam)
print(c_result)

[1. 2. 1. 1. 3. 2. 1. 1. 1. 2.]
{0: 5.0}


In [1]:
def flow_minicost2(G,K,alpa,bet,gam,delt):
    #Minimizing the cost of fractional multicomodity flow

    from sage.numerical.mip import MixedIntegerLinearProgram, MIPSolverException

    # initialize a minimization program
    p = MixedIntegerLinearProgram(maximization=False)
    Kk = range(len(K))
    sk = np.zeros(10)
    tk = np.zeros(10)
    bk = np.zeros(10)
    for k in range(0,10):
        sk[k], tk[k], bk[k] = K[k]
    print(bk)
    # declaring variables
    x = p.new_variable(real= True, nonnegative = True, name='x')
    z = p.new_variable(binary=True, name='z') #if the arc is used at least one request
    y = p.new_variable(binary=True, name = 'y') #a pair of arcs uv and vu is used
    c = p.new_variable(integer = True, nonnegative = True, name = 'c') #capacity
    w = p.new_variable(real = True, nonnegative = True, name = 'w')

    # add constraints:
    for k in range(0,10):
        for u, v, label in G.edges():
            p.add_constraint(x[u,v,k] <= z[u,v]*100)
    for u,v, label in G.edges():
        p.add_constraint(0<= z[u,v] + z[v,u] - 2*y[u,v])
        p.add_constraint(z[u,v] + z[v,u] - 2*y[u,v] <=1)
    #for u, v, label in G.edges():
        #p.add_constraint(sum(sum(x[u,v,k] for u, v, label in G.edges())) for k in K == t[u,v])
    for k in Kk:
        for u in G.vertices():
            if (u==sk[k]):
                p.add_constraint(sum(x[u,v,k] for v in G.neighbors(u)) - sum(x[v,u,k] for v in G.neighbors(u)) == bk[k])
            elif (u==tk[k]):
                p.add_constraint(sum(x[u,v,k] for v in G.neighbors(u)) - sum(x[v,u,k] for v in G.neighbors(u)) == -bk[k])
            else:
                p.add_constraint(sum(x[u,v,k] for v in G.neighbors(u)) - sum(x[v,u,k] for v in G.neighbors(u)) == 0)
    for u,v, label in G.edges():
        p.add_constraint(sum(x[u,v,k] for k in Kk) <= c[0])

    for u in G.vertices():
        for k in Kk:
            if(u == tk[k]):
                p.add_constraint(w[u,k] == 0)
            else:
                p.add_constraint(w[u,k] == sum(x[u,v,k] for v in G.neighbors(u)))
    #add objective to minimize
    p.set_objective(sum(z[u,v] for u, v, label in G.edges())*alpa + sum(sum(x[u,v,k] for u, v, label in G.edges()) for k in Kk)*bet - sum(y[u,v] for u, v, label in G.edges())*gam*0.5 + sum(sum(w[u,k] for u in G.vertices()) for k in Kk)*delt)
    #p.set_objective(c[0])
    #try:
    p.solve()
    #except MIPSolverException:
        #print("the problem has no feasible solution")
        #return []
    c_result = p.get_values(c)
    return c_result

In [2]:
import numpy as np

G = graphs.CycleGraph(10)
G.add_edges([(1, 3), (4, 9), (5, 8)])
G = DiGraph(G)
K = [(0, 1, 1), (0, 5, 2), (0, 6, 1), (1, 2, 1), (2, 9, 3), (1, 6, 2), (2, 7, 1), (2, 8, 1), (6, 5, 1), (8, 1, 2)]
alpa = 100
bet = 5
gam = 20
delt = 10
c_result = flow_minicost2(G,K,alpa,bet,gam,delt)
print(c_result)

[1. 2. 1. 1. 3. 2. 1. 1. 1. 2.]


{0: 10.0}


The effect of the adding \delta is that we now need to also minimize the quantities of nodes used in the routing traffic.